In [1]:
pip install datasets 

   ---------------------------------------- 0.0/561.5 kB ? eta -:--:--
   ---------------------------------------- 561.5/561.5 kB 4.8 MB/s eta 0:00:00

   -------- ------------------------------- 1/5 [multiprocess]
   -------- ------------------------------- 1/5 [multiprocess]
   -------- ------------------------------- 1/5 [multiprocess]
  Attempting uninstall: fsspec
   -------- ------------------------------- 1/5 [multiprocess]
    Found existing installation: fsspec 2025.3.2
   -------- ------------------------------- 1/5 [multiprocess]
   ---------------- ----------------------- 2/5 [fsspec]
    Uninstalling fsspec-2025.3.2:
   ---------------- ----------------------- 2/5 [fsspec]
      Successfully uninstalled fsspec-2025.3.2
   ---------------- ----------------------- 2/5 [fsspec]
   ---------------- ----------------------- 2/5 [fsspec]
   ---------------- ----------------------- 2/5 [fsspec]
   ---------------- ----------------------- 2/5 [fsspec]
   ---------------- ----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
s3fs 2025.3.2 requires fsspec==2025.3.2.*, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
import re
def sentence_tokenizer(paragraph):
  pattern='[\u0964!?.][^0-9.0-9]'
  sentences=re.split(pattern,paragraph)
  ends=re.findall(pattern,paragraph)
  res=[]
  for sentence in sentences:
    if ends==[]:
      break
    end=ends.pop(0)
    res.append(sentence.strip() + end)
  return res


def word_tokenizer(sentence):
    url_pattern = r'https?://(?:www\.)?[a-zA-Z0-9-]+\.[a-zA-Z]{2,}'
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'
  
    phone_pattern = r'(?:\+?[0-9\u0966-\u096F]{1,3}[\s-]?)?[0-9\u0966-\u096F]{10}'

    # Time format: 10:30 or १०:३०
    time_pattern = r'[0-9\u0966-\u096F]{1,2}:[0-9\u0966-\u096F]{2}'

    # Decimal and whole numbers
    latin_number = r'[0-9]+(?:\.[0-9]+)?'
    devnag_number = r'[\u0966-\u096F]+(?:\.[\u0966-\u096F]+)?'

    # Punctuation
    punctuation = r'[!?\.\u0964\,"\'\+-]'

    # Hindi / Devanagari words
    hindi_word = fr'[\u0900-\u0963\u0965-\u096F]+[^!?\.\u0964\,"\'\+-]'
    english_word = r'[a-zA-Z]+'


    # Emojis (basic unicode emoji range)
    emoji_pattern = r'[\U0001F300-\U0001FAFF\U00002700-\U000027BF]'

    # Final combined pattern
    pattern = fr'{email_pattern}|{url_pattern}|{phone_pattern}|{time_pattern}|{latin_number}|{devnag_number}|{punctuation}|{hindi_word}|{emoji_pattern}|{english_word}'

    return re.findall(pattern, sentence)




In [3]:
from datasets import load_dataset
from tqdm import tqdm
import re
import csv

In [6]:
dataset = load_dataset("ai4bharat/IndicCorpV2", "indiccorp_v2", split="hin_Deva", streaming=True)

In [7]:
dataset

IterableDataset({
    features: Unknown,
    num_shards: 3
})

In [ ]:
tokenized_words_csv = "hindi_tokenized_words2.csv"           
reconstructed_csv = "hindi_reconstructed_sentences2.csv"     
re_sentence_csv = "hindi_re_sentence_tokenized2.csv" 

In [9]:
max_lines = 10_000_000   
log_every = 100_000      
flush_every = 10_000 

In [ ]:
import csv
from tqdm import tqdm

# ---------- File Paths ----------
tokenized_words_csv = "hindi_tokenized_words.csv"
reconstructed_csv = "hindi_reconstructed_sentences.csv"
re_sentence_csv = "hindi_re_sentence.csv"

max_lines = 10_000_000
log_every = 100_000
flush_every = 50_000

with open(tokenized_words_csv, "w", encoding="utf-8", newline='') as words_file, \
     open(reconstructed_csv, "w", encoding="utf-8", newline='') as recon_file, \
     open(re_sentence_csv, "w", encoding="utf-8", newline='') as re_sent_file:

    words_writer = csv.writer(words_file)
    recon_writer = csv.writer(recon_file)
    re_sent_writer = csv.writer(re_sent_file)

    # Headers
    words_writer.writerow(["token"])
    recon_writer.writerow(["reconstructed_sentence"])
    re_sent_writer.writerow(["re_sentence"])

    count_original_sentences = 0
    count_re_sentences = 0

    pbar = tqdm(total=max_lines, desc="Processing", unit="sent")

    for row in dataset:                      
        text = row.get("text", "") or ""
        sentences = sentence_tokenizer(text)  

        for sent in sentences:
            #  word-tokenize this sentence
            tokens = word_tokenizer(sent)

            #  individual tokens (one per row)
            for token in tokens:
                words_writer.writerow([token])

            # reconstructed sentence (space-joined)
            reconstructed = " ".join(tokens)
            recon_writer.writerow([reconstructed])

            # sentence-tokenize the reconstructed string
            re_sentences = sentence_tokenizer(reconstructed)
            if re_sentences:
                for rs in re_sentences:
                    re_sent_writer.writerow([rs])
                    count_re_sentences += 1
            else:
                re_sent_writer.writerow([reconstructed])
                count_re_sentences += 1

            count_original_sentences += 1
            pbar.update(1)

            if count_original_sentences % flush_every == 0:
                words_file.flush()
                recon_file.flush()
                re_sent_file.flush()

            if count_original_sentences % log_every == 0:
                print(f"[LOG] original sentences processed: {count_original_sentences:,} | re-sentences written: {count_re_sentences:,}")

            if count_original_sentences >= max_lines:
                break

        if count_original_sentences >= max_lines:
            break

    pbar.close()

print(f"Done! Sentences processed: {count_original_sentences:,}, Re-sentences: {count_re_sentences:,}")
print(f"Files saved:\n - {tokenized_words_csv}\n - {reconstructed_csv}\n - {re_sentence_csv}")


Processed original sentences:   1%|▍                                    | 103222/10000000 [00:45<16:29, 10003.54sent/s]

[LOG] original sentences processed: 100,000 | re-sentences written: 101,167


Processed original sentences:   2%|▊                                    | 203208/10000000 [01:05<10:28, 15592.18sent/s]

[LOG] original sentences processed: 200,000 | re-sentences written: 202,345


Processed original sentences:   3%|█                                    | 302431/10000000 [01:21<10:22, 15573.74sent/s]

...

[LOG] original sentences processed: 9,800,000 | re-sentences written: 9,923,074


Processed original sentences:  99%|███████████████████████████████████▋| 9901662/10000000 [33:20<00:06, 15440.00sent/s]

[LOG] original sentences processed: 9,900,000 | re-sentences written: 10,024,427


Processed original sentences: 100%|████████████████████████████████████| 10000000/10000000 [33:33<00:00, 4966.85sent/s]

[LOG] original sentences processed: 10,000,000 | re-sentences written: 10,125,710


In [1]:
import pandas as pd

In [3]:
df=pd.read_csv('hindi_tokenized_words2.csv')

In [4]:
df.shape

(190001440, 1)

In [8]:
df.sample(10)

,token
73366781,हुए
137130712,वे
86120992,रीयूनियन
61333423,से
100765490,तो
148551164,मोर्टार
137683431,और
28023666,हिंसा
80981008,15
85803718,पश्चात


In [9]:
df.describe()

,token
count,190001334
unique,1402691
top,।
freq,7822780


In [10]:
df_sen_re=pd.read_csv('hindi_reconstructed_sentences2.csv')

In [11]:
df_sen_re.shape

(10000000, 1)

In [12]:
df_sen_re.sample(20)

,reconstructed_sentence
4928966,इसीलिए वहां के हालात इतने भयावह हो गये ...
7389105,वहीं एक अन्य घटना में उसी तालुका के को...
4906085,गूंगी - बहरी विधवा महिला करीब बीस दिन से...
8996197,नर्मदा तट तिलवाराघाट से भक्तों की टोली ...
1753037,सीएम ने कहा कि सोमवार को पीएम के साथ ...
712829,समुद्धि महाराज ने कहा कि मंगलवार को महा...
4667536,वैसे जल्द ही दीया अपने बच्चे को जन्म द...
8594323,ऐसे में मनाली पुलिस ने बीमारी से हुई म...
8890955,"उधर , वरुण ने निर्वाचन आयोग के नोटिस का ..."
4081493,"व्यापार मंडल अध्यक्ष विवेक अग्रवाल , राजेन..."


In [18]:
df_sen_re.iloc[7389105].reconstructed_sentence

'वहीं  एक  अन्य  घटना  में  उसी  तालुका  के  कोडिम्बाला  गांव  के  निवासी  70 वर्षीय  एक  व्यक्ति  ने  एक  पेड़  से  लटक  कर  कथित  तौर  पर  आत्महत्या  कर  ली .'

In [20]:
df_sen_re.describe()

,reconstructed_sentence
count,10000000
unique,9479602
top,.
freq,44424


In [19]:
df_sen_cn=pd.read_csv('hindi_re_sentence_tokenized2.csv')

In [32]:
df_sen_cn.iloc[8].re_sentence

'हम  इसे  जितना  सरल  रखेंगे , उतना  ही  बेहतर  होगा .'

In [34]:
df_words = pd.read_csv("hindi_tokenized_words2.csv") 
df_sentences = pd.read_csv("hindi_reconstructed_sentences2.csv")
df_words.head()
df_sentences.head()

,reconstructed_sentence
0,इनेलो 1987 में उस वक्त ऐसे ही दोराहे पर...
1,हालांकि तब पार्टी पर देवीलाल की मजबूत प...
2,1989 में देवीलाल केन्द्र की राजनीति में ...
3,उन परिस्थितियों में देवीलाल ने कड़ा निर्...
4,उस समय रणजीत की नाराजगी के चलते उनके स...


In [39]:
print(df_words.columns)
import string

Index(['token'], dtype='object')


In [ ]:
total_sentences = len(df_sentences)

total_words = len(df_words)

total_chars = df_words['token'].str.len().sum()

avg_sentence_length = total_words / total_sentences

avg_word_length = total_chars / total_words

punctuations = set(string.punctuation) | set(['।', '’', '‘', '“', '”'])  
tokens_no_punc = df_words[~df_words['token'].isin(punctuations)]
unique_tokens_no_punc = tokens_no_punc['token'].nunique()
ttr_no_punc = unique_tokens_no_punc / total_words

# Print
print(f"Total sentences: {total_sentences:,}")
print(f"Total words: {total_words:,}")
print(f"Total characters (no spaces): {total_chars:,}")
print(f"Average sentence length: {avg_sentence_length:.2f} words/sentence")
print(f"Average word length: {avg_word_length:.2f} characters/word")
print(f"Type/Token Ratio (no punctuation in TTR): {ttr_no_punc:.4f}")

Total sentences: 10,000,000
Total words: 190,001,440
Total characters (no spaces): 863,708,929.0
Average sentence length: 19.00 words/sentence
Average word length: 4.55 characters/word
Type/Token Ratio (no punctuation in TTR): 0.0074
